In [ ]:
!pip install -U transformers bitsandbytes gradio accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import textwrap

# Initialize device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "Venkatesh20/healthbot",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)
tokenizer = AutoTokenizer.from_pretrained("Venkatesh20/healthbot")

def format_response(response):
    """Format the response for better readability"""
    response = ' '.join(response.split())  # Remove duplicate whitespace
    return textwrap.fill(response, width=80)  # Wrap text

def get_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=200)
    raw_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return format_response(raw_response.replace(prompt, '')).strip()

# Updated CSS with better visibility
css = """
.gradio-container {
    max-width: 800px;
    margin: auto;
    font-family: 'Segoe UI', Arial, sans-serif;
}
.message.user {
    background: #f0f7ff;
    padding: 12px 16px;
    border-radius: 8px;
    margin: 8px 0;
    border-left: 4px solid #1e88e5;
}
.message.bot {
    background: #f9f9f9;
    padding: 12px 16px;
    border-radius: 8px;
    margin: 8px 0;
    border-left: 4px solid #4caf50;
    white-space: pre-wrap;
    color: #000000 !important;
}
.message.user, .message.bot, .message.user *:not(a), .message.bot *:not(a){
  color: #000000 !important;
}
.chatbot .wrap{
  color: #000000 !important;
}
.disclaimer-box {
    background: #fff3e0;
    padding: 12px 16px;
    border-radius: 8px;
    margin-top: 16px;
    border-left: 4px solid #ffa000;
    font-size: 0.9em;
    color: #000000;
}
#component-0, #component-1, #component-2 {
    padding: 12px 0;
}
h1 {

    border-bottom: 1px solid #eee;
    padding-bottom: 12px;
}
/* Ensure all text in chat is black */
.chatbot {
    color: #000000 !important;
}
/* Text input box */
/* Make sure examples are visible */
.examples {
    color: #000000 !important;
}
"""

with gr.Blocks(css=css, theme=gr.themes.Default()) as demo:
    gr.Markdown("""
    # 🩺 HealthBot Assistant
    *Your trusted source for general health information*
    """)

    chatbot = gr.Chatbot(label="Conversation", height=400)
    msg = gr.Textbox(label="", placeholder="Type your health question here...",
                   lines=2, max_lines=4)

    with gr.Row():
        submit_btn = gr.Button("Ask", variant="primary")
        clear_btn = gr.Button("Clear Conversation", variant="secondary")

    gr.Examples(
        examples=[
            "What are common causes of tooth pain?",
            "How to relieve a headache naturally?",
            "When should I worry about a fever in adults?",
            "What are the early signs of diabetes?"
        ],
        inputs=msg,
        label="Common Questions"
    )

    with gr.Accordion("ℹ️ Important Disclaimer", open=False):
        gr.Markdown("""
        <div class="disclaimer-box">
        This AI provides general health information only. It is not a substitute for
        professional medical advice, diagnosis, or treatment. Always consult your
        healthcare provider for specific medical concerns.
        </div>
        """)

    def respond(message, chat_history):
        if not message.strip():
            return "", chat_history
        try:
            bot_message = get_response(message)
            chat_history.append((message, bot_message))
            return "", chat_history
        except Exception as e:
            error_msg = "I encountered an error. Please rephrase your question."
            chat_history.append((message, error_msg))
            return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    submit_btn.click(respond, [msg, chatbot], [msg, chatbot])
    clear_btn.click(lambda: None, None, chatbot, queue=False)

demo.launch()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/810 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/572 [00:00<?, ?B/s]

<ipython-input-3-6892dc9dc319>:90: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation", height=400)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2829cebc7d760cccb1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
